In [1]:
import sys
import os
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.model_selection import train_test_split 
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

import pickle
from IPython.display import clear_output
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.5
K.tensorflow_backend.set_session(tf.Session(config=config))

# Experiment 1

* pickle1.p
* Marker = einzelne Channel + Mono
* Random Mono-Snippets
* getMeanMFCC() x 40 (untereinander)
* Architektur: CNN von https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

In [220]:
features = pickle.load(open("Pickles/pickle1.p", "rb"))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

,feature,class_label
0,"[-272.93866, 78.29002, -5.7069273, 12.786187, ...",klapp
950,"[-605.18494, 94.289894, -12.569014, 18.861961,...",klapp
1900,"[-409.41293, 122.481445, -41.31422, 19.439909,...",klapp
2850,"[-602.84, 65.77663, -3.561166, 43.207966, -8.1...",noklapp
3800,"[-542.72516, 172.78352, -28.462677, 27.740074,...",noklapp
4750,"[-427.63205, 72.44588, 10.115246, 27.494013, -...",noklapp


In [221]:
features_cp = []
for f,c in features:
    features_cp.append([ [[f],]*40, c ])
    
features_cp[12][0][0:2:]

[[array([-2.4183440e+02,  7.5498520e+01, -8.5412226e+00,  8.3656340e+00,
         -4.5100298e+00,  3.9749825e+00, -1.0074351e+00,  1.6417691e+00,
         -1.9899381e+00,  2.0210383e+00, -1.7172641e+00,  2.8858922e+00,
         -1.5965629e+00,  2.4626105e+00,  1.0557063e+00,  2.3589902e+00,
         -3.9865655e-01,  2.6551561e+00, -3.8521993e-01,  1.9522010e+00,
         -3.0949333e-01,  2.6241479e+00,  1.2644924e+00,  2.8620899e+00,
         -2.7649137e-01,  1.4117364e+00,  2.2772765e-01,  1.6094500e+00,
          1.0077388e+00,  2.0562439e+00,  6.8004107e-01,  1.0299205e+00,
         -2.7629113e-01,  5.7498223e-01,  4.9020937e-01,  1.3876550e+00,
          4.7108080e-02,  3.8061407e-01, -5.4256582e-01,  1.0440761e+00],
        dtype=float32)],
 [array([-2.4183440e+02,  7.5498520e+01, -8.5412226e+00,  8.3656340e+00,
         -4.5100298e+00,  3.9749825e+00, -1.0074351e+00,  1.6417691e+00,
         -1.9899381e+00,  2.0210383e+00, -1.7172641e+00,  2.8858922e+00,
         -1.5965629e+00, 

In [222]:
featuresdf = pd.DataFrame(features_cp, columns=['feature','class_label'])
featuresdf[::950]

,feature,class_label
0,"[[[-272.93866, 78.29002, -5.7069273, 12.786187...",klapp
950,"[[[-605.18494, 94.289894, -12.569014, 18.86196...",klapp
1900,"[[[-409.41293, 122.481445, -41.31422, 19.43990...",klapp
2850,"[[[-602.84, 65.77663, -3.561166, 43.207966, -8...",noklapp
3800,"[[[-542.72516, 172.78352, -28.462677, 27.74007...",noklapp
4750,"[[[-427.63205, 72.44588, 10.115246, 27.494013,...",noklapp


In [223]:
# Convert features and corresponding classification labels into numpy arrays
X = np.swapaxes(np.array(featuresdf.feature.tolist()), 2,3)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [226]:
num_rows = 40 #40
num_columns = 40 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))


# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 39, 39, 16)        80        
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 19, 19, 16)        0         
_________________________________________________________________
dropout_65 (Dropout)         (None, 19, 19, 16)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 18, 18, 32)        2080      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 9, 9, 32)          0         
_________________________________________________________________
dropout_66 (Dropout)         (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 8, 8, 64)          8256      
__________

In [227]:
num_epochs = 100
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=0, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=0)

clear_output()
duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training completed in time:  0:00:50.693866
Training Accuracy:  0.8569929152609825
Testing Accuracy:  0.8121720888932379


In [228]:
num_epochs = 50
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=0, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=0)

clear_output()
duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training completed in time:  0:01:03.642409
Training Accuracy:  0.9092101810704815
Testing Accuracy:  0.8205666324399326


# Experiment 2

* Marker = einzelne Channel + Mono
* Random Mono-Snippets
* getMeanMFCC()
* Architektur: von https://towardsdatascience.com/how-to-apply-machine-learning-and-deep-learning-methods-to-audio-analysis-615e286fcbbc

In [115]:
features = pickle.load(open("Pickles/pickle1.p", "rb"))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

,feature,class_label
0,"[-272.93866, 78.29002, -5.7069273, 12.786187, ...",klapp
950,"[-605.18494, 94.289894, -12.569014, 18.861961,...",klapp
1900,"[-409.41293, 122.481445, -41.31422, 19.439909,...",klapp
2850,"[-602.84, 65.77663, -3.561166, 43.207966, -8.1...",noklapp
3800,"[-542.72516, 172.78352, -28.462677, 27.740074,...",noklapp
4750,"[-427.63205, 72.44588, 10.115246, 27.494013, -...",noklapp


In [116]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [117]:
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 127)

In [28]:
num_labels = yy.shape[1]
filter_size = 2
input_shape=(1,40)

model = Sequential()
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.build(input_shape)
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]
print("Pre-training accuracy: %.4f%%" % accuracy)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (1, 256)                  10496     
_________________________________________________________________
activation_28 (Activation)   (1, 256)                  0         
_________________________________________________________________
dropout_19 (Dropout)         (1, 256)                  0         
_________________________________________________________________
dense_29 (Dense)             (1, 256)                  65792     
_________________________________________________________________
activation_29 (Activation)   (1, 256)                  0         
_________________________________________________________________
dropout_20 (Dropout)         (1, 256)                  0         
_________________________________________________________________
dense_30 (Dense)             (1, 2)                    514       
__________

In [29]:
num_epochs = 100
num_batch_size = 32
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)

Train on 3811 samples, validate on 953 samples
Epoch 1/100


InvalidArgumentError: Inputs to operation training_1/Adam/gradients/loss_4/activation_30_loss/clip_by_value_grad/Select of type Select must have the same size and shape.  Input 0: [32,2] != input 2: [1,2]
	 [[{{node training_1/Adam/gradients/loss_4/activation_30_loss/clip_by_value_grad/Select}}]]

# Experiment 3

* Marker = einzelne Channel + Mono
* Random Mono-Snippets
* getMFCC()
* Architektur: CNN von https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

In [2]:
features = pickle.load(open("Pickles/pickle2.p", "rb"))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

,feature,class_label
0,"[[-401.91074, -364.88733, -400.43365, -425.726...",klapp
950,"[[-705.9435, -687.03906, -656.64685, -627.5050...",klapp
1900,"[[-465.21176, -459.15497, -468.00946, -471.744...",klapp
2850,"[[-611.73395, -593.7865, -598.858, -603.34607,...",noklapp
3800,"[[-429.78662, -431.02716, -436.34952, -435.642...",noklapp
4750,"[[-364.97586, -353.12744, -347.85767, -353.366...",noklapp


In [3]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [4]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

W0319 09:59:22.148208 140192084776768 deprecation_wrapper.py:119] From /home/mf151/.direnv/python-3.7.3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0319 09:59:22.161520 140192084776768 deprecation_wrapper.py:119] From /home/mf151/.direnv/python-3.7.3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0319 09:59:22.163864 140192084776768 deprecation_wrapper.py:119] From /home/mf151/.direnv/python-3.7.3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0319 09:59:22.176325 140192084776768 deprecation_wrapper.py:119] From /home/mf151/.direnv/python-3.7.3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Pleas

In [5]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

W0319 09:59:27.987616 140192084776768 deprecation_wrapper.py:119] From /home/mf151/.direnv/python-3.7.3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0319 09:59:28.010834 140192084776768 deprecation_wrapper.py:119] From /home/mf151/.direnv/python-3.7.3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 39, 93, 16)        80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 46, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 46, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 45, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 22, 32)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 22, 32)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 21, 64)         8256      
__________

In [6]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

W0319 09:59:42.942686 140192084776768 deprecation.py:323] From /home/mf151/.direnv/python-3.7.3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3800 samples, validate on 951 samples
Epoch 1/72
3800/3800 [==============================] - 2s 435us/step - loss: 2.0006 - acc: 0.5550 - val_loss: 0.6759 - val_acc: 0.5636

Epoch 00001: val_loss improved from inf to 0.67589, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
3800/3800 [==============================] - 1s 342us/step - loss: 0.7552 - acc: 0.6182 - val_loss: 0.5990 - val_acc: 0.6646

Epoch 00002: val_loss improved from 0.67589 to 0.59902, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
3800/3800 [==============================] - 1s 343us/step - loss: 0.5779 - acc: 0.7003 - val_loss: 0.4980 - val_acc: 0.7907

Epoch 00003: val_loss improved from 0.59902 to 0.49803, saving model to weights.best.basic_cnn.hdf5
Epoch 4/72
3800/3800 [==============================] - 1s 340us/step - loss: 0.4808 - acc: 0.7776 - val_loss: 0.4300 - val_acc: 0.8349

Epoch 00004: val_loss improved from 0.49803 to 0.42998, saving model to weights.best.basic_cnn.hdf5
Epoch 5/7

3800/3800 [==============================] - 1s 342us/step - loss: 0.0745 - acc: 0.9737 - val_loss: 0.0676 - val_acc: 0.9842

Epoch 00040: val_loss did not improve from 0.05722
Epoch 41/72
3800/3800 [==============================] - 1s 341us/step - loss: 0.0674 - acc: 0.9792 - val_loss: 0.0522 - val_acc: 0.9884

Epoch 00041: val_loss improved from 0.05722 to 0.05217, saving model to weights.best.basic_cnn.hdf5
Epoch 42/72
3800/3800 [==============================] - 1s 341us/step - loss: 0.0727 - acc: 0.9755 - val_loss: 0.0831 - val_acc: 0.9832

Epoch 00042: val_loss did not improve from 0.05217
Epoch 43/72
3800/3800 [==============================] - 1s 340us/step - loss: 0.0716 - acc: 0.9750 - val_loss: 0.0638 - val_acc: 0.9842

Epoch 00043: val_loss did not improve from 0.05217
Epoch 44/72
3800/3800 [==============================] - 1s 341us/step - loss: 0.0697 - acc: 0.9750 - val_loss: 0.0892 - val_acc: 0.9800

Epoch 00044: val_loss did not improve from 0.05217
Epoch 45/72
3800/3

In [9]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9947368421052631
Testing Accuracy:  0.9894847528916929


In [10]:
model.save('weights.model')
model.save_weights("model.h5")
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Experiment 4

* Marker = einzelne Channel + Mono => augmented position
* Random Mono-Snippets
* getMFCC()
* Architektur: CNN von https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

In [3]:
features = pickle.load(open("Pickles/pickle3.p", "rb"))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

,feature,class_label
0,"[[-401.91074, -364.88733, -400.43365, -425.726...",klapp
950,"[[-560.4174, -548.76294, -535.0601, -538.0909,...",klapp
1900,"[[-546.99133, -537.48883, -538.3186, -539.6693...",klapp
2850,"[[-546.2755, -549.9481, -560.5604, -550.44867,...",klapp
3800,"[[-705.9435, -687.03906, -656.64685, -627.5050...",klapp
4750,"[[-420.05396, -422.36597, -434.14792, -440.575...",klapp
5700,"[[-518.5563, -516.15, -525.5188, -525.7212, -5...",klapp
6650,"[[-738.4582, -738.41833, -734.59064, -693.3244...",klapp
7600,"[[-577.9406, -578.9035, -586.558, -593.03253, ...",klapp
8550,"[[-582.8322, -582.97144, -599.4791, -608.0502,...",klapp


In [4]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [8]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 39, 93, 16)        80        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 46, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 19, 46, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 45, 32)        2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 22, 32)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 9, 22, 32)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 21, 64)         8256      
__________

In [10]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 15232 samples, validate on 3808 samples
Epoch 1/72
15232/15232 [==============================] - 2s 145us/step - loss: 1.1532 - acc: 0.6226 - val_loss: 0.4726 - val_acc: 0.8393

Epoch 00001: val_loss improved from inf to 0.47258, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
15232/15232 [==============================] - 2s 125us/step - loss: 0.4279 - acc: 0.8184 - val_loss: 0.3673 - val_acc: 0.8939

Epoch 00002: val_loss improved from 0.47258 to 0.36730, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
15232/15232 [==============================] - 2s 124us/step - loss: 0.3381 - acc: 0.8712 - val_loss: 0.3033 - val_acc: 0.9123

Epoch 00003: val_loss improved from 0.36730 to 0.30332, saving model to weights.best.basic_cnn.hdf5
Epoch 4/72
15232/15232 [==============================] - 2s 124us/step - loss: 0.3152 - acc: 0.8836 - val_loss: 0.2986 - val_acc: 0.9152

Epoch 00004: val_loss improved from 0.30332 to 0.29858, saving model to weights.best.basic_cnn.hdf5

In [11]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])
!telegram-send "server = fertig"

Training Accuracy:  0.9810924369747899
Testing Accuracy:  0.9477415966386554


In [12]:
model.save('Models/m1/weights.model')
model.save_weights("Models/m1/model.h5")
model_json = model.to_json()
with open("Models/m1/model.json", "w") as json_file:
    json_file.write(model_json)

# Experiment 5

* Marker = einzelne Channel + Mono => augmented position
* Mono-Snippets von Maxima
* getMFCC()
* Architektur: CNN von https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

In [7]:
features = pickle.load(open("Pickles/pickle4.p", "rb"))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

,feature,class_label
0,"[[-401.91074, -364.88733, -400.43365, -425.726...",klapp
950,"[[-498.68527, -499.05252, -511.57742, -512.935...",noklapp
1900,"[[-459.3803, -435.96304, -426.47766, -421.5678...",noklapp
2850,"[[-155.8534, -152.13937, -165.1922, -188.13705...",noklapp
3800,"[[-580.1882, -579.0551, -560.98486, -523.3863,...",klapp
4750,"[[-570.72217, -578.7941, -598.3212, -607.0586,...",klapp
5700,"[[-449.0259, -443.26535, -443.9801, -455.3299,...",noklapp
6650,"[[-624.7864, -624.66296, -634.1999, -633.471, ...",klapp
7600,"[[-635.62146, -600.41113, -578.12714, -560.429...",noklapp
8550,"[[-331.40616, -327.403, -324.71365, -310.99008...",klapp


In [8]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
!telegram-send "Server: Features konvertiert"

In [15]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = 2#yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [16]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 39, 93, 16)        80        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 19, 46, 16)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 19, 46, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 18, 45, 32)        2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 9, 22, 32)         0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 9, 22, 32)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 21, 64)         8256      
__________

In [17]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 11208 samples, validate on 2802 samples
Epoch 1/72
11208/11208 [==============================] - 2s 155us/step - loss: 2.4008 - acc: 0.5927 - val_loss: 0.6342 - val_acc: 0.5921

Epoch 00001: val_loss improved from inf to 0.63415, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
11208/11208 [==============================] - 1s 124us/step - loss: 0.5299 - acc: 0.7217 - val_loss: 0.4960 - val_acc: 0.7698

Epoch 00002: val_loss improved from 0.63415 to 0.49599, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
11208/11208 [==============================] - 1s 126us/step - loss: 0.4752 - acc: 0.7625 - val_loss: 0.4644 - val_acc: 0.8133

Epoch 00003: val_loss improved from 0.49599 to 0.46437, saving model to weights.best.basic_cnn.hdf5
Epoch 4/72
11208/11208 [==============================] - 1s 124us/step - loss: 0.4317 - acc: 0.7911 - val_loss: 0.3801 - val_acc: 0.8640

Epoch 00004: val_loss improved from 0.46437 to 0.38009, saving model to weights.best.basic_cnn.hdf5

11208/11208 [==============================] - 1s 125us/step - loss: 0.1244 - acc: 0.9532 - val_loss: 0.1098 - val_acc: 0.9607

Epoch 00039: val_loss did not improve from 0.09453
Epoch 40/72
11208/11208 [==============================] - 1s 124us/step - loss: 0.1318 - acc: 0.9499 - val_loss: 0.1060 - val_acc: 0.9636

Epoch 00040: val_loss did not improve from 0.09453
Epoch 41/72
11208/11208 [==============================] - 1s 122us/step - loss: 0.1203 - acc: 0.9546 - val_loss: 0.1155 - val_acc: 0.9622

Epoch 00041: val_loss did not improve from 0.09453
Epoch 42/72
11208/11208 [==============================] - 1s 124us/step - loss: 0.1162 - acc: 0.9578 - val_loss: 0.1091 - val_acc: 0.9600

Epoch 00042: val_loss did not improve from 0.09453
Epoch 43/72
11208/11208 [==============================] - 1s 125us/step - loss: 0.1164 - acc: 0.9569 - val_loss: 0.1070 - val_acc: 0.9604

Epoch 00043: val_loss did not improve from 0.09453
Epoch 44/72
11208/11208 [==============================] 

In [18]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])
!telegram-send "Server: Habe alles evaluiert"

Training Accuracy:  0.9908101356174162
Testing Accuracy:  0.9689507492944907


In [19]:
model.save('Models/m3/weights.model')
model.save_weights("Models/m3/model.h5")
model_json = model.to_json()
with open("Models/m3/model.json", "w") as json_file:
    json_file.write(model_json)

# Experiment 5 - Part 2

* Marker = einzelne Channel + Mono => augmented position
* Mono-Snippets von Maxima
* getMFCC()
* Architektur: CNN von https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

=> VERBESSERTE DATEN, KEINE FEHLER MEHR

In [28]:
features = pickle.load(open("Pickles/pickle4-1.p", "rb"))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

,feature,class_label
0,"[[-401.91074, -364.88733, -400.43365, -425.726...",klapp
950,"[[-529.0275, -530.5678, -539.6287, -504.5892, ...",klapp
1900,"[[-611.8759, -607.2732, -600.64984, -602.05817...",klapp
2850,"[[-549.2779, -537.589, -535.74164, -534.98505,...",noklapp
3800,"[[-355.80963, -372.6017, -409.63934, -429.1113...",klapp
4750,"[[-663.6253, -660.92163, -667.2111, -665.5719,...",klapp
5700,"[[-583.66675, -575.15375, -569.56854, -551.033...",noklapp
6650,"[[-503.17474, -503.21194, -509.01248, -523.404...",noklapp
7600,"[[-601.5306, -597.71576, -598.6808, -594.0496,...",noklapp
8550,"[[-291.71292, -300.8574, -311.50537, -264.5054...",noklapp


In [35]:
len(features)

15199

In [29]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
!telegram-send "Server: Features konvertiert"

In [30]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = 2#yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [31]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 39, 93, 16)        80        
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 19, 46, 16)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 19, 46, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 18, 45, 32)        2080      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 9, 22, 32)         0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 9, 22, 32)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 21, 64)         8256      
__________

In [32]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 12159 samples, validate on 3040 samples
Epoch 1/72
12159/12159 [==============================] - 2s 157us/step - loss: 0.7930 - acc: 0.6318 - val_loss: 0.5136 - val_acc: 0.7605

Epoch 00001: val_loss improved from inf to 0.51356, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
12159/12159 [==============================] - 1s 121us/step - loss: 0.4799 - acc: 0.7598 - val_loss: 0.3544 - val_acc: 0.8533

Epoch 00002: val_loss improved from 0.51356 to 0.35437, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
12159/12159 [==============================] - 1s 121us/step - loss: 0.3871 - acc: 0.8217 - val_loss: 0.3773 - val_acc: 0.8046

Epoch 00003: val_loss did not improve from 0.35437
Epoch 4/72
12159/12159 [==============================] - 1s 122us/step - loss: 0.3428 - acc: 0.8467 - val_loss: 0.3053 - val_acc: 0.8747

Epoch 00004: val_loss improved from 0.35437 to 0.30527, saving model to weights.best.basic_cnn.hdf5
Epoch 5/72
12159/12159 [========================

12159/12159 [==============================] - 1s 123us/step - loss: 0.0998 - acc: 0.9613 - val_loss: 0.1073 - val_acc: 0.9582

Epoch 00041: val_loss did not improve from 0.09924
Epoch 42/72
12159/12159 [==============================] - 1s 118us/step - loss: 0.1061 - acc: 0.9580 - val_loss: 0.0920 - val_acc: 0.9674

Epoch 00042: val_loss improved from 0.09924 to 0.09198, saving model to weights.best.basic_cnn.hdf5
Epoch 43/72
12159/12159 [==============================] - 1s 122us/step - loss: 0.1003 - acc: 0.9613 - val_loss: 0.0862 - val_acc: 0.9691

Epoch 00043: val_loss improved from 0.09198 to 0.08624, saving model to weights.best.basic_cnn.hdf5
Epoch 44/72
12159/12159 [==============================] - 1s 123us/step - loss: 0.0965 - acc: 0.9617 - val_loss: 0.0797 - val_acc: 0.9711

Epoch 00044: val_loss improved from 0.08624 to 0.07968, saving model to weights.best.basic_cnn.hdf5
Epoch 45/72
12159/12159 [==============================] - 1s 121us/step - loss: 0.1007 - acc: 0.9604

In [33]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])
!telegram-send "Server: Habe alles evaluiert"

Training Accuracy:  0.9934205115552266
Testing Accuracy:  0.9726973684210526


In [26]:
os.mkdir(os.path.join(os.getcwd(), "Models", "m4"))

In [34]:
model.save('Models/m4/weights.model')
model.save_weights("Models/m4/model.h5")
model_json = model.to_json()
with open("Models/m4/model.json", "w") as json_file:
    json_file.write(model_json)

# Experiment 5 - Part 3

* Marker = einzelne Channel + Mono => augmented position
* Mono-Snippets von Maxima
* getMFCC()
* Architektur: CNN von https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

=> VERBESSERTE DATEN, KEINE FEHLER MEHR

In [38]:
features = pickle.load(open("Pickles/pickle4-2.p", "rb"))
print("len(features) = ", len(features))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

len(features) =  15222


,feature,class_label
0,"[[-552.32306, -539.6811, -537.4932, -546.9935,...",klapp
950,"[[-504.08615, -497.9757, -487.82037, -492.8520...",klapp
1900,"[[-701.8161, -693.2406, -688.5021, -684.5822, ...",noklapp
2850,"[[-546.99133, -537.48883, -538.3186, -539.6693...",klapp
3800,"[[-716.00964, -701.72455, -692.82227, -694.809...",noklapp
4750,"[[-626.96893, -629.7167, -639.7845, -654.0444,...",klapp
5700,"[[-849.5224, -838.67224, -832.0527, -824.084, ...",klapp
6650,"[[-359.46896, -355.5226, -362.19086, -360.8022...",klapp
7600,"[[-574.2854, -599.0054, -650.21686, -676.9735,...",noklapp
8550,"[[-453.0503, -455.63123, -489.18375, -518.4922...",noklapp


In [39]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
!telegram-send "Server: Features konvertiert"

In [40]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [41]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 39, 93, 16)        80        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 19, 46, 16)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 19, 46, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 18, 45, 32)        2080      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 9, 22, 32)         0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 9, 22, 32)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 21, 64)         8256      
__________

In [42]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 12177 samples, validate on 3045 samples
Epoch 1/72
12177/12177 [==============================] - 3s 206us/step - loss: 1.3860 - acc: 0.6114 - val_loss: 0.5775 - val_acc: 0.7172

Epoch 00001: val_loss improved from inf to 0.57751, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
12177/12177 [==============================] - 1s 118us/step - loss: 0.5392 - acc: 0.7101 - val_loss: 0.4727 - val_acc: 0.7934

Epoch 00002: val_loss improved from 0.57751 to 0.47268, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
12177/12177 [==============================] - 1s 117us/step - loss: 0.4692 - acc: 0.7643 - val_loss: 0.3968 - val_acc: 0.8397

Epoch 00003: val_loss improved from 0.47268 to 0.39680, saving model to weights.best.basic_cnn.hdf5
Epoch 4/72
12177/12177 [==============================] - 1s 115us/step - loss: 0.4212 - acc: 0.7989 - val_loss: 0.3346 - val_acc: 0.8617

Epoch 00004: val_loss improved from 0.39680 to 0.33457, saving model to weights.best.basic_cnn.hdf5

Epoch 39/72
12177/12177 [==============================] - 1s 115us/step - loss: 0.1215 - acc: 0.9519 - val_loss: 0.1273 - val_acc: 0.9537

Epoch 00039: val_loss did not improve from 0.10809
Epoch 40/72
12177/12177 [==============================] - 1s 116us/step - loss: 0.1170 - acc: 0.9526 - val_loss: 0.1136 - val_acc: 0.9560

Epoch 00040: val_loss did not improve from 0.10809
Epoch 41/72
12177/12177 [==============================] - 1s 116us/step - loss: 0.1119 - acc: 0.9553 - val_loss: 0.0904 - val_acc: 0.9685

Epoch 00041: val_loss improved from 0.10809 to 0.09042, saving model to weights.best.basic_cnn.hdf5
Epoch 42/72
12177/12177 [==============================] - 1s 115us/step - loss: 0.1218 - acc: 0.9521 - val_loss: 0.1112 - val_acc: 0.9603

Epoch 00042: val_loss did not improve from 0.09042
Epoch 43/72
12177/12177 [==============================] - 1s 117us/step - loss: 0.1102 - acc: 0.9584 - val_loss: 0.1199 - val_acc: 0.9537

Epoch 00043: val_loss did not improve from 0.09

In [43]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.988256549232159
Testing Accuracy:  0.9707717569786535


In [44]:
os.mkdir(os.path.join(os.getcwd(), "Models", "m5"))

In [45]:
model.save('Models/m5/weights.model')
model.save_weights("Models/m5/model.h5")
model_json = model.to_json()
with open("Models/m5/model.json", "w") as json_file:
    json_file.write(model_json)

# Experiment 7

* Marker = einzelne Channel + Mono + random mixes => augmented position
* Mono- und Random-Mix-Snippets von Maxima (mehr verschiedene Snippets, nicht positionsaugmentiert)
* getMFCC()
* Architektur: CNN von https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

In [3]:
features = pickle.load(open("Pickles/pickle6.p", "rb"))
print("len(features) = ", len(features))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::950]

len(features) =  14394


,feature,class_label
0,"[[-552.32306, -539.6811, -537.4932, -546.9935,...",klapp
950,"[[-504.08615, -497.9757, -487.82037, -492.8520...",klapp
1900,"[[-701.8161, -693.2406, -688.5021, -684.5822, ...",noklapp
2850,"[[-546.99133, -537.48883, -538.3186, -539.6693...",klapp
3800,"[[-716.00964, -701.72455, -692.82227, -694.809...",noklapp
4750,"[[-626.96893, -629.7167, -639.7845, -654.0444,...",klapp
5700,"[[-849.5224, -838.67224, -832.0527, -824.084, ...",klapp
6650,"[[-359.46896, -355.5226, -362.19086, -360.8022...",klapp
7600,"[[-574.2854, -599.0054, -650.21686, -676.9735,...",noklapp
8550,"[[-453.0503, -455.63123, -489.18375, -518.4922...",noklapp


In [4]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
!telegram-send "Server: Features konvertiert"

In [56]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) # -> sigmoid

In [57]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 39, 93, 16)        80        
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 19, 46, 16)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 19, 46, 16)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 18, 45, 32)        2080      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 9, 22, 32)         0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 9, 22, 32)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 8, 21, 64)         8256      
__________

In [58]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 11515 samples, validate on 2879 samples
Epoch 1/72
11515/11515 [==============================] - 2s 174us/step - loss: 1.3467 - acc: 0.6324 - val_loss: 0.5778 - val_acc: 0.7371

Epoch 00001: val_loss improved from inf to 0.57775, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
11515/11515 [==============================] - 1s 123us/step - loss: 0.5529 - acc: 0.7112 - val_loss: 0.5198 - val_acc: 0.7197

Epoch 00002: val_loss improved from 0.57775 to 0.51977, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
11515/11515 [==============================] - 1s 124us/step - loss: 0.4737 - acc: 0.7618 - val_loss: 0.4494 - val_acc: 0.7906

Epoch 00003: val_loss improved from 0.51977 to 0.44942, saving model to weights.best.basic_cnn.hdf5
Epoch 4/72
11515/11515 [==============================] - 1s 125us/step - loss: 0.4102 - acc: 0.8076 - val_loss: 0.3491 - val_acc: 0.8628

Epoch 00004: val_loss improved from 0.44942 to 0.34913, saving model to weights.best.basic_cnn.hdf5

In [59]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9889709075119409
Testing Accuracy:  0.970475859362949


In [60]:
folder = "m7"
os.mkdir(os.path.join(os.getcwd(), "Models", folder))

In [61]:
model.save('Models/'+folder+'/weights.model')
model.save_weights("Models/"+folder+"/model.h5")
model_json = model.to_json()
with open("Models/"+folder+"/model.json", "w") as json_file:
    json_file.write(model_json)

# MobileNet

In [65]:
from keras.applications import MobileNet

model = MobileNet(weights=None, input_shape=(num_rows, num_columns, num_channels), classes=2, include_top=True)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 40, 94, 1)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 41, 95, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 20, 47, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 20, 47, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 20, 47, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 20, 47, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 20, 47, 32)        128       
__________

In [66]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 11515 samples, validate on 2879 samples
Epoch 1/72
11515/11515 [==============================] - 17s 1ms/step - loss: 0.5097 - acc: 0.7548 - val_loss: 0.9591 - val_acc: 0.7544

Epoch 00001: val_loss improved from inf to 0.95909, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
11515/11515 [==============================] - 7s 623us/step - loss: 0.2819 - acc: 0.8763 - val_loss: 0.3678 - val_acc: 0.8628

Epoch 00002: val_loss improved from 0.95909 to 0.36783, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
11515/11515 [==============================] - 7s 613us/step - loss: 0.1614 - acc: 0.9375 - val_loss: 0.2197 - val_acc: 0.9271

Epoch 00003: val_loss improved from 0.36783 to 0.21974, saving model to weights.best.basic_cnn.hdf5
Epoch 4/72
11515/11515 [==============================] - 7s 622us/step - loss: 0.0953 - acc: 0.9653 - val_loss: 0.2192 - val_acc: 0.9333

Epoch 00004: val_loss improved from 0.21974 to 0.21922, saving model to weights.best.basic_cnn.hdf5



Epoch 00040: val_loss did not improve from 0.02614
Epoch 41/72
11515/11515 [==============================] - 7s 622us/step - loss: 0.0263 - acc: 0.9924 - val_loss: 0.0444 - val_acc: 0.9913

Epoch 00041: val_loss did not improve from 0.02614
Epoch 42/72
11515/11515 [==============================] - 7s 608us/step - loss: 0.0099 - acc: 0.9970 - val_loss: 0.0512 - val_acc: 0.9899

Epoch 00042: val_loss did not improve from 0.02614
Epoch 43/72
11515/11515 [==============================] - 7s 617us/step - loss: 0.0099 - acc: 0.9964 - val_loss: 0.0562 - val_acc: 0.9844

Epoch 00043: val_loss did not improve from 0.02614
Epoch 44/72
11515/11515 [==============================] - 7s 620us/step - loss: 0.0085 - acc: 0.9979 - val_loss: 0.0457 - val_acc: 0.9885

Epoch 00044: val_loss did not improve from 0.02614
Epoch 45/72
11515/11515 [==============================] - 7s 622us/step - loss: 0.0046 - acc: 0.9987 - val_loss: 0.0243 - val_acc: 0.9934

Epoch 00045: val_loss improved from 0.02614 

In [67]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9994789405123752
Testing Accuracy:  0.9961792288989232


In [68]:
folder = "m8_mobilenet"
os.mkdir(os.path.join(os.getcwd(), "Models", folder))

In [69]:
model.save('Models/'+folder+'/weights.model')
model.save_weights("Models/"+folder+"/model.h5")
model_json = model.to_json()
with open("Models/"+folder+"/model.json", "w") as json_file:
    json_file.write(model_json)

# MobileNet V2

In [75]:
from keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(weights=None, input_shape=(num_rows, num_columns, num_channels), classes=2, include_top=True)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 40, 94, 1)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 41, 95, 1)    0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 20, 47, 32)   288         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 20, 47, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [76]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 11515 samples, validate on 2879 samples
Epoch 1/72
11515/11515 [==============================] - 20s 2ms/step - loss: 0.4223 - acc: 0.7937 - val_loss: 2.1439 - val_acc: 0.7457

Epoch 00001: val_loss improved from inf to 2.14393, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
11515/11515 [==============================] - 10s 853us/step - loss: 0.1467 - acc: 0.9450 - val_loss: 6.3776 - val_acc: 0.5346

Epoch 00002: val_loss did not improve from 2.14393
Epoch 3/72
11515/11515 [==============================] - 10s 860us/step - loss: 0.0761 - acc: 0.9739 - val_loss: 0.2978 - val_acc: 0.9535

Epoch 00003: val_loss improved from 2.14393 to 0.29780, saving model to weights.best.basic_cnn.hdf5
Epoch 4/72
11515/11515 [==============================] - 10s 846us/step - loss: 0.0413 - acc: 0.9866 - val_loss: 4.4458 - val_acc: 0.6419

Epoch 00004: val_loss did not improve from 0.29780
Epoch 5/72
11515/11515 [==============================] - 10s 869us/step - loss: 0.0330 - acc: 

11515/11515 [==============================] - 10s 864us/step - loss: 0.0061 - acc: 0.9977 - val_loss: 0.7551 - val_acc: 0.9357

Epoch 00042: val_loss did not improve from 0.12245
Epoch 43/72
11515/11515 [==============================] - 10s 872us/step - loss: 0.0084 - acc: 0.9974 - val_loss: 0.7450 - val_acc: 0.9403

Epoch 00043: val_loss did not improve from 0.12245
Epoch 44/72
11515/11515 [==============================] - 10s 858us/step - loss: 0.0068 - acc: 0.9979 - val_loss: 0.6886 - val_acc: 0.9458

Epoch 00044: val_loss did not improve from 0.12245
Epoch 45/72
11515/11515 [==============================] - 10s 859us/step - loss: 0.0084 - acc: 0.9974 - val_loss: 0.4433 - val_acc: 0.9587

Epoch 00045: val_loss did not improve from 0.12245
Epoch 46/72
11515/11515 [==============================] - 10s 871us/step - loss: 0.0066 - acc: 0.9983 - val_loss: 0.1272 - val_acc: 0.9885

Epoch 00046: val_loss did not improve from 0.12245
Epoch 47/72
11515/11515 [===========================

In [77]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9906209292227529
Testing Accuracy:  0.9826328583209206


In [73]:
folder = "m9_mobilenet2"
os.mkdir(os.path.join(os.getcwd(), "Models", folder))

In [78]:
model.save('Models/'+folder+'/weights.model')
model.save_weights("Models/"+folder+"/model.h5")
model_json = model.to_json()
with open("Models/"+folder+"/model.json", "w") as json_file:
    json_file.write(model_json)

# Densenet121

In [11]:
from keras.applications.densenet import DenseNet121
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1
num_labels = yy.shape[1]
filter_size = 2
model = 0
model = DenseNet121(weights=None, input_shape=(num_rows, num_columns, num_channels), classes=2, include_top=True)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40, 94, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 46, 100, 1)   0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 20, 47, 64)   3136        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 20, 47, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [12]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 11515 samples, validate on 2879 samples
Epoch 1/72
11515/11515 [==============================] - 39s 3ms/step - loss: 0.1655 - acc: 0.9357 - val_loss: 0.8182 - val_acc: 0.7506

Epoch 00001: val_loss improved from inf to 0.81820, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
11515/11515 [==============================] - 20s 2ms/step - loss: 0.0725 - acc: 0.9734 - val_loss: 0.1083 - val_acc: 0.9653

Epoch 00002: val_loss improved from 0.81820 to 0.10830, saving model to weights.best.basic_cnn.hdf5
Epoch 3/72
11515/11515 [==============================] - 19s 2ms/step - loss: 0.0415 - acc: 0.9865 - val_loss: 0.2893 - val_acc: 0.9062

Epoch 00003: val_loss did not improve from 0.10830
Epoch 4/72
11515/11515 [==============================] - 19s 2ms/step - loss: 0.0380 - acc: 0.9871 - val_loss: 0.2779 - val_acc: 0.9291

Epoch 00004: val_loss did not improve from 0.10830
Epoch 5/72
11515/11515 [==============================] - 19s 2ms/step - loss: 0.0252 - acc: 0.9921 -


Epoch 00040: val_loss did not improve from 0.00035
Epoch 41/72
11515/11515 [==============================] - 19s 2ms/step - loss: 0.0037 - acc: 0.9987 - val_loss: 0.0265 - val_acc: 0.9927

Epoch 00041: val_loss did not improve from 0.00035
Epoch 42/72
11515/11515 [==============================] - 19s 2ms/step - loss: 0.0043 - acc: 0.9989 - val_loss: 0.0052 - val_acc: 0.9986

Epoch 00042: val_loss did not improve from 0.00035
Epoch 43/72
11515/11515 [==============================] - 19s 2ms/step - loss: 2.1462e-04 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 0.9990

Epoch 00043: val_loss did not improve from 0.00035
Epoch 44/72
11515/11515 [==============================] - 19s 2ms/step - loss: 1.1955e-04 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 0.9990

Epoch 00044: val_loss did not improve from 0.00035
Epoch 45/72
11515/11515 [==============================] - 20s 2ms/step - loss: 1.1438e-04 - acc: 1.0000 - val_loss: 0.0031 - val_acc: 0.9993

Epoch 00045: val_loss did not improve

In [13]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  1.0
Testing Accuracy:  0.9993053143452588


In [14]:
folder = "m10_densenet121"
os.mkdir(os.path.join(os.getcwd(), "Models", folder))

In [15]:
model.save('Models/'+folder+'/weights.model')
model.save_weights("Models/"+folder+"/model.h5")
model_json = model.to_json()
with open("Models/"+folder+"/model.json", "w") as json_file:
    json_file.write(model_json)

# NASNetMobile

In [20]:
from keras.applications.nasnet import NASNetMobile
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1
num_labels = yy.shape[1]
filter_size = 2
model = 0
model = NASNetMobile(weights=None, input_shape=(num_rows, num_columns, num_channels), classes=2, include_top=True)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 40, 94, 1)    0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 19, 46, 32)   288         input_5[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 19, 46, 32)   128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_377 (Activation)     (None, 19, 46, 32)   0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

In [21]:
num_epochs = 3
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Train on 11515 samples, validate on 2879 samples
Epoch 1/3
11515/11515 [==============================] - 108s 9ms/step - loss: 0.3197 - acc: 0.8562 - val_loss: 0.6688 - val_acc: 0.7840

Epoch 00001: val_loss improved from inf to 0.66885, saving model to weights.best.basic_cnn.hdf5
Epoch 2/3
11515/11515 [==============================] - 37s 3ms/step - loss: 0.1153 - acc: 0.9568 - val_loss: 2.3795 - val_acc: 0.6367

Epoch 00002: val_loss did not improve from 0.66885
Epoch 3/3
11515/11515 [==============================] - 37s 3ms/step - loss: 0.0565 - acc: 0.9797 - val_loss: 0.1268 - val_acc: 0.9677

Epoch 00003: val_loss improved from 0.66885 to 0.12682, saving model to weights.best.basic_cnn.hdf5
Training completed in time:  0:17:32.408061


In [22]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9754233608336952
Testing Accuracy:  0.9676971170545329


In [23]:
folder = "m11_nasnetmobile"
os.mkdir(os.path.join(os.getcwd(), "Models", folder))

In [24]:
model.save('Models/'+folder+'/weights.model')
model.save_weights("Models/"+folder+"/model.h5")
model_json = model.to_json()
with open("Models/"+folder+"/model.json", "w") as json_file:
    json_file.write(model_json)

# More Data with MNv2

In [3]:
features = pickle.load(open("Pickles/pickle_new.p", "rb"))
print("len(features) = ", len(features))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::1000]

len(features) =  44107


,feature,class_label
0,"[[-489.44638, -487.61813, -491.89145, -487.434...",klapp
1000,"[[-583.7778, -572.5055, -571.0877, -572.32904,...",klapp
2000,"[[-648.4932, -645.98413, -643.8437, -645.07465...",klapp
3000,"[[-706.79474, -712.2376, -725.35156, -719.0223...",klapp
4000,"[[-704.52527, -697.11365, -694.676, -696.69824...",klapp
5000,"[[-733.2682, -723.3961, -725.324, -724.75464, ...",klapp
6000,"[[-400.89813, -398.52722, -406.01337, -407.850...",klapp
7000,"[[-602.1073, -598.3384, -596.61847, -598.0705,...",klapp
8000,"[[-294.3977, -293.51813, -298.5968, -314.58716...",klapp
9000,"[[-333.1444, -337.65695, -349.35815, -349.3763...",klapp


In [4]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
!telegram-send "Server: Features konvertiert"

In [6]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

In [7]:
from keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(weights=None, input_shape=(num_rows, num_columns, num_channels), classes=2, include_top=True)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 94, 1)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 41, 95, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 20, 47, 32)   288         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 20, 47, 32)   128         Conv1[0][0]                      
_____________________________________

In [8]:
num_epochs = 72
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Instructions for updating:
Use tf.cast instead.
Train on 35285 samples, validate on 8822 samples
Epoch 1/72
35285/35285 [==============================] - 37s 1ms/step - loss: 0.2640 - acc: 0.8813 - val_loss: 0.3338 - val_acc: 0.9525

Epoch 00001: val_loss improved from inf to 0.33383, saving model to weights.best.basic_cnn.hdf5
Epoch 2/72
35285/35285 [==============================] - 30s 855us/step - loss: 0.0513 - acc: 0.9824 - val_loss: 0.5086 - val_acc: 0.9442

Epoch 00002: val_loss did not improve from 0.33383
Epoch 3/72
35285/35285 [==============================] - 31s 866us/step - loss: 0.0345 - acc: 0.9888 - val_loss: 0.5123 - val_acc: 0.9363

Epoch 00003: val_loss did not improve from 0.33383
Epoch 4/72
35285/35285 [==============================] - 31s 866us/step - loss: 0.0300 - acc: 0.9901 - val_loss: 0.1302 - val_acc: 0.9835

Epoch 00004: val_loss improved from 0.33383 to 0.13022, saving model to weights.best.basic_cnn.hdf5
Epoch 5/72
35285/35285 [=======================

In [9]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9996599121439705
Testing Accuracy:  0.9980729993198821


In [10]:
folder = "m12"
os.mkdir(os.path.join(os.getcwd(), "Models", folder))

In [11]:
model.save('Models/'+folder+'/weights.model')
model.save_weights("Models/"+folder+"/model.h5")
model_json = model.to_json()
with open("Models/"+folder+"/model.json", "w") as json_file:
    json_file.write(model_json)

# More Data with MNv2, for CoreML

In [3]:
features = pickle.load(open("Pickles/pickle_new2.p", "rb"))
print("len(features) = ", len(features))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
featuresdf[::2000]

len(features) =  38358


,feature,class_label
0,"[[-489.44638, -487.61813, -491.89145, -487.434...",klapp
2000,"[[-559.9284, -548.13257, -542.2534, -549.5109,...",klapp
4000,"[[-547.9967, -539.6625, -532.66705, -524.167, ...",klapp
6000,"[[-774.3586, -767.3389, -759.1228, -756.4714, ...",klapp
8000,"[[-440.1031, -439.75232, -448.6616, -456.94064...",klapp
10000,"[[-430.08942, -421.84375, -427.56442, -431.628...",klapp
12000,"[[-512.4252, -502.7536, -489.03235, -496.12973...",klapp
14000,"[[-542.21643, -553.39233, -573.47577, -581.750...",klapp
16000,"[[-541.2151, -535.37646, -521.6501, -509.19128...",klapp
18000,"[[-520.8307, -506.76318, -504.6328, -512.44025...",klapp


In [4]:
# Convert features and corresponding classification labels into numpy arrays
X = [np.expand_dims(features[0][0], axis=2)] #[] #np.array(featuresdf.feature.tolist())
for f in features[1::]:
    xi = np.expand_dims(f[0], axis=2)
    X = np.append(X, [xi], axis=0)
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
!telegram-send "Server: Features konvertiert"

In [5]:
num_rows = 40 #40
num_columns = 94 #1
num_channels = 1

#x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
#x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

In [6]:
from keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(weights=None, input_shape=(num_rows, num_columns, num_channels), classes=2, include_top=True)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 94, 1)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 41, 95, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 20, 47, 32)   288         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 20, 47, 32)   128         Conv1[0][0]                      
_____________________________________

In [7]:
num_epochs = 50
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)
!telegram-send "Server: Habe alles gelernt"

Instructions for updating:
Use tf.cast instead.
Train on 30686 samples, validate on 7672 samples
Epoch 1/50
30686/30686 [==============================] - 33s 1ms/step - loss: 0.2602 - acc: 0.8820 - val_loss: 0.3159 - val_acc: 0.9562

Epoch 00001: val_loss improved from inf to 0.31587, saving model to weights.best.basic_cnn.hdf5
Epoch 2/50
30686/30686 [==============================] - 26s 849us/step - loss: 0.0484 - acc: 0.9836 - val_loss: 0.4185 - val_acc: 0.9472

Epoch 00002: val_loss did not improve from 0.31587
Epoch 3/50
30686/30686 [==============================] - 26s 862us/step - loss: 0.0297 - acc: 0.9896 - val_loss: 0.1205 - val_acc: 0.9859

Epoch 00003: val_loss improved from 0.31587 to 0.12052, saving model to weights.best.basic_cnn.hdf5
Epoch 4/50
30686/30686 [==============================] - 26s 858us/step - loss: 0.0223 - acc: 0.9921 - val_loss: 0.2944 - val_acc: 0.9691

Epoch 00004: val_loss did not improve from 0.12052
Epoch 5/50
30686/30686 [=======================


Epoch 00040: val_loss did not improve from 0.01376
Epoch 41/50
30686/30686 [==============================] - 26s 859us/step - loss: 0.0018 - acc: 0.9993 - val_loss: 0.0129 - val_acc: 0.9974

Epoch 00041: val_loss improved from 0.01376 to 0.01293, saving model to weights.best.basic_cnn.hdf5
Epoch 42/50
30686/30686 [==============================] - 26s 860us/step - loss: 0.0038 - acc: 0.9989 - val_loss: 0.9851 - val_acc: 0.9059

Epoch 00042: val_loss did not improve from 0.01293
Epoch 43/50
30686/30686 [==============================] - 27s 865us/step - loss: 0.0033 - acc: 0.9989 - val_loss: 0.0391 - val_acc: 0.9936

Epoch 00043: val_loss did not improve from 0.01293
Epoch 44/50
30686/30686 [==============================] - 26s 860us/step - loss: 0.0029 - acc: 0.9992 - val_loss: 0.0266 - val_acc: 0.9952

Epoch 00044: val_loss did not improve from 0.01293
Epoch 45/50
30686/30686 [==============================] - 26s 860us/step - loss: 0.0046 - acc: 0.9986 - val_loss: 0.0614 - val_acc

In [8]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9907123769525364
Testing Accuracy:  0.9889207507820647


In [9]:
folder = "m13_coreml"
os.mkdir(os.path.join(os.getcwd(), "Models", folder))

In [10]:
model.save('Models/'+folder+'/weights.model')
model.save_weights("Models/"+folder+"/model.h5")
model_json = model.to_json()
with open("Models/"+folder+"/model.json", "w") as json_file:
    json_file.write(model_json)

Für CoreML:

In [12]:
model.save('Models/'+folder+'/weights_only.h5')